In [133]:
import requests
import polars as pl
from io import StringIO

api = "https://download.bls.gov/pub/time.series"
endpoints = {
    "ConsumerPriceApparelData": "cu/cu.data.2.Summaries",
    "ConsumerPriceHousingData": "cu/cu.data.12.USHousing",
    "ConsumerPriceMedicalData": "cu/cu.data.15.USMedical",
    "ConsumerPriceRecreationData": "cu/cu.data.16.USRecreation",
    "ProducerPriceData": "wp/wp.data.22.FD-ID",
}
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}

series_ids = {
    "ConsumerPriceApparelData": {
        "CUUR0000SAA": "Apparel (NSA)",
        "CUSR0000SAA": "Apparel (SA)",
    },
    "ConsumerPriceHousingData": {
        "CUUR0000SAH3": "Household Furnishings and Operations (NSA)",
        "CUSR0000SAH3": "Household Furnishings and Operations (SA)",
    },
    "ConsumerPriceMedicalData": {
        "CUUR0000SEMC01": "Physicians' Services (NSA)",
        "CUSR0000SEMC01": "Physicians' Services (SA)",
        "CUUR0000SEMD01": "Hospital Services (NSA)",
        "CUSR0000SEMD01": "Hospital Services (SA)",
        "CUUR0000SAM1": "Medical Care Commodities (NSA)",
        "CUSR0000SAM1": "Medical Care Commodities (SA)",
    },
    "ConsumerPriceRecreationData": {
        "CUUR0000SAR": "Recreation (NSA)",
        "CUSR0000SAR": "Recreation (SA)",
    },
    "ProducerPriceData": {
        "WPUFD49208": "Finished Goods Less Energy (NSA)",
        "WPSFD49208": "Finished Goods Less Energy (SA)",
        "WPUFD4131": "Finished Goods Less Food and Energy (NSA)",
        "WPSFD4131": "Finished Goods Less Food and Energy (SA)",
    },
}

def generate_raw(cat: str):
    curr_srs = series_ids[cat]

    raw_df = df.filter(1==1) \
        .filter(
            (pl.col('period').str.starts_with('S')==False) # [S01, S02, S03]. not sure exactly what these are.
            & (pl.col('period') != 'M13') # Month 13 is the average of that series for that full calendar year.
            & (pl.col('series_id').is_in(list(curr_srs.keys())))
        ) \
        .with_columns(
            pl.col('period').str.slice(-2).cast(pl.Int8).alias('month'),
            pl.col('series_id').replace(curr_srs).alias("category"),
        ) \
        .with_columns(
            pl.date(pl.col('year'), pl.col('month'), 1).alias('date')
        ) \
        .select([
            'category',
            'series_id', 
            'date', 
            'value',
        ])
    
    return raw_df

def generate_pivot(raw_df):
    pivot_df = raw_df.filter(1==1) \
        .pivot(on=['series_id'], index=['date'], values=['value']) \
        .sort('date')
    
    return pivot_df

def generate_yr_agg_pivot(pivot_df):
    yr_agg_pivot_df = pivot_df.filter(1==1) \
        .with_columns(
            pl.col('date').dt.year().alias('year'),
        ) \
        .group_by(pl.col('year')) \
        .agg(pl.all().exclude('year', 'date').mean()) \
        .sort('year')

    return yr_agg_pivot_df


categories = endpoints.keys()
for cat in categories:
    url = f"{api}/{endpoints[cat]}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        text="\n".join([" ".join(line.split()) for line in response.text.splitlines()])
        df = pl.read_csv(StringIO(text), separator=" ")
        # print(df)
    except:
        print(response.status_code)

    raw_df = generate_raw(cat)
    pivot_df = generate_pivot(raw_df)
    yr_agg_pivot_df = generate_yr_agg_pivot(pivot_df)
    


In [136]:
pivot_df

date,WPSFD4131,WPSFD49208,WPUFD4131,WPUFD49208
date,f64,f64,f64,f64
1973-01-01,null,null,47.2,null
1973-02-01,null,null,47.4,null
1973-03-01,null,null,47.5,null
1973-04-01,null,null,47.8,null
1973-05-01,null,null,48.0,null
…,…,…,…,…
2025-03-01,257.253,263.639,257.469,263.617
2025-04-01,258.087,263.322,258.254,263.369
2025-05-01,258.915,263.915,258.979,264.057


In [122]:
sorted(df['period'].unique().to_list())

['M01',
 'M02',
 'M03',
 'M04',
 'M05',
 'M06',
 'M07',
 'M08',
 'M09',
 'M10',
 'M11',
 'M12',
 'M13']

In [111]:
cat

'ConsumerPriceMedicalData'

In [124]:
series_ids[cat].keys()

dict_keys(['WPUFD49208', 'WPSFD49208', 'WPUFD4131', 'WPSFD4131'])

In [123]:
curr_srs = series_ids[cat]

raw = df.filter(1==1) \
    .filter(
        (pl.col('period').str.starts_with('S')==False) # [S01, S02, S03]. not sure exactly what these are.
        & (pl.col('period') != 'M13') # Month 13 is the average of that series for that full calendar year.
        & (pl.col('series_id').is_in(list(curr_srs.keys())))
    ) \
    .with_columns(
        pl.col('period').str.slice(-2).cast(pl.Int8).alias('month'),
        pl.col('series_id').replace(curr_srs).alias("category"),
    ) \
    .with_columns(
        pl.date(pl.col('year'), pl.col('month'), 1).alias('date')
    ) \
    .select([
        'category',
        'series_id', 
        'date', 
        'value',
    ])
    
print(raw)

shape: (2_464, 4)
┌─────────────────────────────────┬────────────┬────────────┬─────────┐
│ category                        ┆ series_id  ┆ date       ┆ value   │
│ ---                             ┆ ---        ┆ ---        ┆ ---     │
│ str                             ┆ str        ┆ date       ┆ f64     │
╞═════════════════════════════════╪════════════╪════════════╪═════════╡
│ Finished Goods Less Food and E… ┆ WPSFD4131  ┆ 1974-01-01 ┆ 49.7    │
│ Finished Goods Less Food and E… ┆ WPSFD4131  ┆ 1974-02-01 ┆ 50.0    │
│ Finished Goods Less Food and E… ┆ WPSFD4131  ┆ 1974-03-01 ┆ 50.5    │
│ Finished Goods Less Food and E… ┆ WPSFD4131  ┆ 1974-04-01 ┆ 51.1    │
│ Finished Goods Less Food and E… ┆ WPSFD4131  ┆ 1974-05-01 ┆ 52.2    │
│ …                               ┆ …          ┆ …          ┆ …       │
│ Finished Goods Less Energy (NS… ┆ WPUFD49208 ┆ 2025-03-01 ┆ 263.617 │
│ Finished Goods Less Energy (NS… ┆ WPUFD49208 ┆ 2025-04-01 ┆ 263.369 │
│ Finished Goods Less Energy (NS… ┆ WPUFD49208

In [125]:
pivot = raw.filter(1==1) \
    .pivot(on=['series_id'], index=['date'], values=['value']) \
    .sort('date')

print(pivot)

shape: (631, 5)
┌────────────┬───────────┬────────────┬───────────┬────────────┐
│ date       ┆ WPSFD4131 ┆ WPSFD49208 ┆ WPUFD4131 ┆ WPUFD49208 │
│ ---        ┆ ---       ┆ ---        ┆ ---       ┆ ---        │
│ date       ┆ f64       ┆ f64        ┆ f64       ┆ f64        │
╞════════════╪═══════════╪════════════╪═══════════╪════════════╡
│ 1973-01-01 ┆ null      ┆ null       ┆ 47.2      ┆ null       │
│ 1973-02-01 ┆ null      ┆ null       ┆ 47.4      ┆ null       │
│ 1973-03-01 ┆ null      ┆ null       ┆ 47.5      ┆ null       │
│ 1973-04-01 ┆ null      ┆ null       ┆ 47.8      ┆ null       │
│ 1973-05-01 ┆ null      ┆ null       ┆ 48.0      ┆ null       │
│ …          ┆ …         ┆ …          ┆ …         ┆ …          │
│ 2025-03-01 ┆ 257.253   ┆ 263.639    ┆ 257.469   ┆ 263.617    │
│ 2025-04-01 ┆ 258.087   ┆ 263.322    ┆ 258.254   ┆ 263.369    │
│ 2025-05-01 ┆ 258.915   ┆ 263.915    ┆ 258.979   ┆ 264.057    │
│ 2025-06-01 ┆ 259.67    ┆ 264.542    ┆ 259.65    ┆ 264.903    │
│ 2025-07

In [126]:
yr_agg_pivot = pivot.filter(1==1) \
    .with_columns(
        pl.col('date').dt.year().alias('year'),
    ) \
    .group_by(pl.col('year')) \
    .agg(pl.all().exclude('year', 'date').mean()) \
    .sort('year')

print(yr_agg_pivot)

shape: (53, 5)
┌──────┬────────────┬────────────┬────────────┬────────────┐
│ year ┆ WPSFD4131  ┆ WPSFD49208 ┆ WPUFD4131  ┆ WPUFD49208 │
│ ---  ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ i32  ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞══════╪════════════╪════════════╪════════════╪════════════╡
│ 1973 ┆ null       ┆ null       ┆ 48.1       ┆ null       │
│ 1974 ┆ 53.608333  ┆ null       ┆ 53.616667  ┆ null       │
│ 1975 ┆ 59.741667  ┆ 62.466667  ┆ 59.725     ┆ 62.433333  │
│ 1976 ┆ 63.116667  ┆ 64.825     ┆ 63.1       ┆ 64.8       │
│ 1977 ┆ 66.883333  ┆ 68.533333  ┆ 66.9       ┆ 68.558333  │
│ …    ┆ …          ┆ …          ┆ …          ┆ …          │
│ 2021 ┆ 218.40125  ┆ 220.4455   ┆ 218.390667 ┆ 220.471917 │
│ 2022 ┆ 236.498333 ┆ 242.1205   ┆ 236.493833 ┆ 242.19875  │
│ 2023 ┆ 247.215917 ┆ 251.318667 ┆ 247.212333 ┆ 251.273083 │
│ 2024 ┆ 253.050333 ┆ 257.92825  ┆ 253.0475   ┆ 257.899583 │
│ 2025 ┆ 258.134286 ┆ 264.260286 ┆ 258.285857 ┆ 264.266143 │
└──────┴─

In [22]:
df1 = df.filter(1==1) \
    .filter(
        (pl.col('period').str.starts_with('S')==False)
        & (pl.col('period') != 'M13')
    ) \
    .with_columns(
        pl.col('period').str.slice(-2).cast(pl.Int8)
    ) \
    .with_columns(
        pl.date(pl.col('year'), pl.col('period'), 1).alias('date'),
    ) \
    .with_columns(
        pl.col('date').dt.strftime("%B").alias('MonthName'),
        pl.col('period')#.cast(pl.String)
    ) \
    .filter(
        (1==1)
        & (pl.col('date') >= pl.date(1975, 1, 1)) 
        & (pl.col('series_id').is_in(['CUUR0000SAA','CUSR0000SAA']))
    ) \
    .group_by(['series_id', 'year', 'period', 'date', 'MonthName']) \
    .agg(pl.mean('value')) \
    .sort(['series_id', 'year', 'period']) \
    # .pivot(on='series_id', index=['year', 'period', 'date', 'MonthName'], values='value') \
    # .select(['year', 'period', 'date', 'MonthName', 'CUUR0000SAA', 'CUSR0000SAA'])

df1

series_id,year,period,date,MonthName,value
str,i64,i8,date,str,f64
"""CUSR0000SAA""",1975,1,1975-01-01,"""January""",71.8
"""CUSR0000SAA""",1975,2,1975-02-01,"""February""",72.0
"""CUSR0000SAA""",1975,3,1975-03-01,"""March""",72.1
"""CUSR0000SAA""",1975,4,1975-04-01,"""April""",72.1
"""CUSR0000SAA""",1975,5,1975-05-01,"""May""",72.2
…,…,…,…,…,…
"""CUUR0000SAA""",2025,3,2025-03-01,"""March""",134.082
"""CUUR0000SAA""",2025,4,2025-04-01,"""April""",132.544
"""CUUR0000SAA""",2025,5,2025-05-01,"""May""",131.223


In [ ]:
df1.apply

In [3]:
df2 = df.filter(1==1) \
    .filter(
        (pl.col('period').str.starts_with('S')==False)
        & (pl.col('period') == 'M13')
    ) \
    .with_columns(
        pl.col('period').str.slice(-2).cast(pl.Int8)
    ) \
    .filter(
        (pl.col('year') >= 1975)
        & (pl.col('series_id').is_in(['CUUR0000SAA','CUSR0000SAA']))
    ) \
    .with_columns(
        (pl.col('value')*0).alias('CUSR0000SAA'),
        pl.col('value').alias('CUUR0000SAA'),
        # pl.col('period').replace_strict(13, 'Annual'),
        pl.lit(None).alias('date'),
        pl.lit(None).alias('MonthName'),
    ) \
    .select(['year', 'period', 'date', 'MonthName', 'CUUR0000SAA', 'CUSR0000SAA'])
    # .select(['year', 'period', 'CUUR0000SAA', 'CUSR0000SAA'])

df2

year,period,date,MonthName,CUUR0000SAA,CUSR0000SAA
i64,i8,null,null,f64,f64
1975,13,null,null,72.5,0.0
1976,13,null,null,75.2,0.0
1977,13,null,null,78.6,0.0
1978,13,null,null,81.4,0.0
1979,13,null,null,84.9,0.0
…,…,…,…,…,…
2020,13,null,null,118.079,0.0
2021,13,null,null,120.993,0.0
2022,13,null,null,127.081,0.0


In [6]:
df3 = pl.concat([df1, df2])
df3.columns = ['Year', 'MonthNum', 'M/YYYY', 'Month', 'Not Seasonally Adjusted', 'Seasonally Adjusted']
df3 = df3.filter(1==1) \
    .sort(['Year', 'MonthNum']) \
    .with_columns(
        pl.col('MonthNum').alias('Period'),
        pl.col('M/YYYY').dt.strftime('%m-%Y'),
        pl.col('MonthNum').cast(pl.String).replace({'13': 'Annual'})

    )

In [21]:
df3.filter(1==1) \
    .filter(pl.col('Year')==1975) \
    .select('Not Seasonally Adjusted') \
    .mean()
    # .sum('Not Seasonally Adjusted')

Not Seasonally Adjusted
f64
72.523077
